# Install Dependencies

In [1]:
%%capture
!pip install monai
!pip install dicom2nifti

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp /content/drive/MyDrive/Machine-Learning-Biomedicine/PankVision-3D/preprocess/final_preprocess.py /content
!cp /content/drive/MyDrive/Machine-Learning-Biomedicine/PankVision-3D/utilities/final_utilities.py /content
!cp /content/drive/MyDrive/Machine-Learning-Biomedicine/PankVision-3D/model/get_model.py /content

In [5]:
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.losses import DiceLoss, DiceCELoss

import torch
from final_utilities import train
from get_model import get_model
from final_preprocess import preprocess_image, prepare

import numpy as np
from skimage import exposure
from monai.transforms import (
    Compose,
    LoadImaged,
    AddChanneld,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
    Resized,
    ToTensord,
)
from monai.data import CacheDataset, Dataset, DataLoader
from glob import glob
import os
from monai.utils import set_determinism

## Checking for CUDA

In [ ]:
if torch.cuda.is_available():
    print("CUDA is available. You can use the GPU.")
else:
    print("CUDA is not available. You can only use the CPU.")

CUDA is available. You can use the GPU.


## Preprocess Dataset

In [6]:
data_dir = '/content/drive/MyDrive/Machine-Learning-Biomedicine/PankVision-3D/dataset/dataset-007/Data_Train_Test/'

In [8]:
data_in = prepare(data_dir, cache=True)

Loading dataset: 100%|██████████| 56/56 [02:02<00:00,  2.18s/it]


# Training

In [10]:
model_dir = '/content/drive/MyDrive/Machine-Learning-Biomedicine/PankVision-3D/results/dataset_007/v7'

In [11]:
import torch
from torch import nn
from monai.losses import DiceLoss

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

args = {
    'model_name': 'UNet',
    'pretrained': True,
    'dropout': 0.1
}
model = get_model(args)
model = model.to(device)

# loss_function = nn.CrossEntropyLoss().to(device)
loss_function = DiceLoss(to_onehot_y=True, softmax=True).to(device)
optimizer = torch.optim.Adam(model.parameters(), 1e-4, amsgrad=True)

if __name__ == '__main__':
    train(model, data_in, loss_function, optimizer, 150, model_dir)

----------
epoch 1/150
1/222, Train_loss: 0.7149
Train_dice: 0.4074
2/222, Train_loss: 0.7107
Train_dice: 0.4125
3/222, Train_loss: 0.7104
Train_dice: 0.4116
4/222, Train_loss: 0.7114
Train_dice: 0.4088
5/222, Train_loss: 0.7055
Train_dice: 0.4174
6/222, Train_loss: 0.7060
Train_dice: 0.4147
7/222, Train_loss: 0.7073
Train_dice: 0.4117
8/222, Train_loss: 0.7051
Train_dice: 0.4132
9/222, Train_loss: 0.7045
Train_dice: 0.4125
10/222, Train_loss: 0.7034
Train_dice: 0.4119
11/222, Train_loss: 0.7020
Train_dice: 0.4126
12/222, Train_loss: 0.6987
Train_dice: 0.4164
13/222, Train_loss: 0.6943
Train_dice: 0.4195


KeyboardInterrupt: ignored

## Check the Size of Data (Add to main) - For Debugging

In [ ]:
# assuming prepare function has been imported from another file
train_loader, test_loader = prepare(data_dir)

for batch_data in  train_loader:
  volume = batch_data["vol"]
  label = batch_data["seg"]

  # print size of volume and label tensors
  print(f"Volume size: {volume.size()}")
  print(f"Label size: {label.size()}")

  # only process one batch
  break